In [ ]:
# !unzip data.zip

In [2]:
# !pip3 install --upgrade pandas

In [197]:
import warnings
warnings.filterwarnings('ignore')

# Входные данные

In [8]:
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

PATH_TO_DATA = 'data'

In [4]:
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
# df_train.dropna(axis=1, how='all', inplace=True)
df['target'] = labels

In [5]:
df.shape

(40000, 231)

Отделим hold-out dataset

In [72]:
df_train, df_ho = train_test_split(df, test_size=5000)

In [73]:
df_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,target
26327,NaN,NaN,NaN,NaN,NaN,5341.0,21.0,NaN,NaN,NaN,NaN,NaN,5064.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3544.0,4430.0,NaN,86.0,792.0,NaN,NaN,210.00,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,4044540.0,NaN,NaN,...,1qmr4RQxul,RO12,NaN,taul,1K8T,487l,vHHY4q6,GHfL3MpXAX,2rMgswy,NaN,gwgA,9_Y1,z5Ry,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,5dnbgcr,NaN,TDceha3,7Zw8,UYBR,FzaX,R4hm94s,oslk,oSyMpgG,LM8l689qOp,NaN,ELof,FSa2,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
10192,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,voArjIXt5a,RO12,NaN,taul,1K8T,7ALb,0qCl1hh,9YuuPSz3BI,NaN,NaN,EVMP,9_Y1,xQ2A,VpdQ,NaN,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,7WwCtIM,LusX,cJvF,FzaX,gfMRJnY,oslk,99nURQu,LM8l689qOp,NaN,NaN,5Acm,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
39407,NaN,NaN,NaN,NaN,NaN,15176.0,7.0,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1064.0,1330.0,NaN,8.0,728.0,NaN,NaN,220.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3455724.0,NaN,NaN,...,2jigUH7ejg,RO12,NaN,taul,1K8T,72yt,eYPJuq4,NFlfMbKIm3,NaN,NaN,Tkse,9_Y1,1Bp0,09_Q,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,kZJtVhC,SssI,UYBR,NaN,42Uv66D,oslk,_BMKSUK,NaN,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
36846,NaN,NaN,NaN,NaN,NaN,49.0,0.0,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,20.0,NaN,0.0,32.0,NaN,NaN,133.76,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,O8Zr4Rdc_a,2Knk1KF,NaN,taul,1K8T,L_LK,PHNvXy8,u2pCocB,qRzZGt8,NaN,hINo,9_Y1,em8I,09_Q,IYzP,NKv3VA1BpP,kIsH,NaN,g5HH,L84s,Ie_5MZs,NaN,5zAG_Cb,NaN,teAY_NX,tlNv,cJvF,FzaX,UF16siJ,zCkv,APgdzOv,LM8l689qOp,NaN,NaN,me1d,6fzt,Zy3gnGM,am7c,NaN,-1
11225,NaN,NaN,NaN,NaN,NaN,5502.0,14.0,NaN,NaN,NaN,NaN,NaN,712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,788.0,985.0,NaN,22.0,592.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,555480.0,NaN,NaN,...,avkq9ayfAo,2Knk1KF,NaN,taul,1K8T,PJLW,HPUGayG,r83_sZi,NaN,NaN,nt0n,9_Y1,vm5R,VpdQ,wMei,me75fM6ugJ,kIsH,NaN,uKAI,L84s,h0lfDKh52u4GP,NaN,NaN,NaN,IoI4mKe,CzUQ,cJvF,FzaX,kTkv10k,oslk,TUP6P29,LM8l689qOp,NaN,NaN,7P5s,RAYp,55YFVY9,mj86,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32086,NaN,NaN,NaN,NaN,NaN,1589.0,7.0,NaN,NaN,NaN,NaN,NaN,1804.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,648.0,810.0,NaN,6.0,352.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,532500.0,NaN,NaN,...,py5rEyXktW,AERks4l,NaN,taul,1K8T,42Cy,PHNvXy8,V6XUlxabJ4VaD,119ls53,NaN,nalJ,9_Y1,SkZj,VpdQ,IYzP,NKv3VA1BpP,kIsH,NaN,uKAI,L84s,Ie_5MZs,NaN,Nj2NBv8,NaN,y5FEvJ5,21c1,cJvF,FzaX,UF16siJ,zCkv,APgdzOv,LM8l689qOp,NaN,ELof,Aoh3,6fzt,xwM2aC7IdeMC0,am7c,NaN,-1
28699,NaN,NaN,NaN,NaN,NaN,616.0,0.0,NaN,NaN,NaN,NaN,NaN,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,195.0,NaN,6.0,56.0,NaN,NaN,166.56,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,95496.0,NaN,NaN,...,4e7gUH7IEC,RO12,SEuy,taul,1K8T,lK27,ho_bxgw,NfN9Bz6HFx,J6hvY4Fk1U,smXZ,bDi2,9_Y1,DmlN,NaN,sYC_,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,ZpHLjE6hJ5,NaN,beK4AFX,FJ51Im9,cJvF,FzaX,AvcOVQC,oslk,r757j0h,LM8l689qOp,NaN,xG3x,w_Ub,RAYp,F2FyR07IdsN7I,mj86,NaN,-1
11502,NaN,NaN,NaN,NaN,NaN,616.0,0.0,NaN,NaN,NaN,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.0,300.0,NaN,4.0,88.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,280

In [64]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

- ROC AUC
- precision, recall, f1, log, accuracy
- 5 фолдов, stratified k-fold, shuffled

In [65]:
cols_all = df_train.columns
cols_nan = df_train.loc[:, df_train.isna().all()].columns
df_num = df_train.select_dtypes(exclude='object')
df_cat = df_train.select_dtypes(include='object')

In [76]:
df_train.dropna(axis=1, how='all', inplace=True)
cols = df_train.columns

In [80]:
%%time
si = SimpleImputer(strategy='most_frequent')
df_train = si.fit_transform(df_train)
df_train = pd.DataFrame(df_train, columns=cols)

CPU times: user 7.71 s, sys: 464 ms, total: 8.17 s
Wall time: 8.17 s


In [82]:
df_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var43,Var44,Var45,Var46,Var47,...,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,target
0,0,0,0,0,0,5341,21,0,0,8,0,5064,0,0,0,0,0,3544,4430,0,86,792,0,0,210,0,5,0,0,15,0,0,4.04454e+06,0,0,0,0,2985.64,0,0,...,0,r__I,1qmr4RQxul,RO12,SEuy,taul,1K8T,487l,vHHY4q6,GHfL3MpXAX,2rMgswy,smXZ,gwgA,9_Y1,z5Ry,VpdQ,IYzP,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,KdSa,5dnbgcr,eGzu,TDceha3,7Zw8,UYBR,FzaX,R4hm94s,oslk,oSyMpgG,LM8l689qOp,4n2X,ELof,FSa2,RAYp,F2FyR07IdsN7I,am7c,-1
1,0,0,0,0,0,0,7,0,0,8,0,0,0,0,45,0,0,0,0,0,0,0,0,0,166.56,0,5,0,0,0,0,0,0,0,0,0,0,2985.64,0,0,...,0,r__I,voArjIXt5a,RO12,SEuy,taul,1K8T,7ALb,0qCl1hh,9YuuPSz3BI,yP09M03,smXZ,EVMP,9_Y1,xQ2A,VpdQ,IYzP,me75fM6ugJ,kIsH,uKAI,Mtgm,NhsEn4L,KdSa,5zARyjR,eGzu,7WwCtIM,LusX,cJvF,FzaX,gfMRJnY,oslk,99nURQu,LM8l689qOp,4n2X,ELof,5Acm,RAYp,F2FyR07IdsN7I,am7c,-1
2,0,0,0,0,0,15176,7,0,0,8,0,56,0,0,0,0,0,1064,1330,0,8,728,0,0,220.08,0,5,0,0,0,0,0,3.45572e+06,0,0,0,0,2985.64,0,0,...,0,r__I,2jigUH7ejg,RO12,SEuy,taul,1K8T,72yt,eYPJuq4,NFlfMbKIm3,yP09M03,smXZ,Tkse,9_Y1,1Bp0,09_Q,zm5i,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,KdSa,5zARyjR,eGzu,kZJtVhC,SssI,UYBR,FzaX,42Uv66D,oslk,_BMKSUK,LM8l689qOp,4n2X,ELof,WqMG,RAYp,F2FyR07IdsN7I,am7c,-1
3,0,0,0,0,0,49,0,0,0,8,0,12,0,0,0,0,0,16,20,0,0,32,0,0,133.76,0,5,0,0,0,0,0,0,0,0,0,0,2985.64,0,0,...,0,r__I,O8Zr4Rdc_a,2Knk1KF,SEuy,taul,1K8T,L_LK,PHNvXy8,u2pCocB,qRzZGt8,smXZ,hINo,9_Y1,em8I,09_Q,IYzP,NKv3VA1BpP,kIsH,g5HH,L84s,Ie_5MZs,KdSa,5zAG_Cb,eGzu,teAY_NX,tlNv,cJvF,FzaX,UF16siJ,zCkv,APgdzOv,LM8l689qOp,4n2X,ELof,me1d,6fzt,Zy3gnGM,am7c,-1
4,0,0,0,0,0,5502,14,0,0,8,0,712,0,0,0,0,0,788,985,0,22,592,0,0,166.56,0,5,0,0,0,0,0,555480,0,0,0,0,2985.64,0,0,...,0,r__I,avkq9ayfAo,2Knk1KF,SEuy,taul,1K8T,PJLW,HPUGayG,r83_sZi,yP09M03,smXZ,nt0n,9_Y1,vm5R,VpdQ,wMei,me75fM6ugJ,kIsH,uKAI,L84s,h0lfDKh52u4GP,KdSa,5zARyjR,eGzu,IoI4mKe,CzUQ,cJvF,FzaX,kTkv10k,oslk,TUP6P29,LM8l689qOp,4n2X,ELof,7P5s,RAYp,55YFVY9,mj86,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0,0,0,0,0,1589,7,0,0,8,0,1804,0,0,0,0,0,648,810,0,6,352,0,0,166.56,0,5,0,0,0,0,0,532500,0,0,0,0,2985.64,0,0,...,0,r__I,py5rEyXktW,AERks4l,SEuy,taul,1K8T,42Cy,PHNvXy8,V6XUlxabJ4VaD,119ls53,smXZ,nalJ,9_Y1,SkZj,VpdQ,IYzP,NKv3VA1BpP,kIsH,uKAI,L84s,Ie_5MZs,KdSa,Nj2NBv8,eGzu,y5FEvJ5,21c1,cJvF,FzaX,UF16siJ,zCkv,APgdzOv,LM8l689qOp,4n2X,ELof,Aoh3,6fzt,xwM2aC7IdeMC0,am7c,-1
34996,0,0,0,0,0,616,0,0,0,8,0,76,0,0,0,0,0,156,195,0,6,56,0,0,166.56,0,5,0,0,0,0,0,95496,0,0,0,0,2985.64,0,0,...,0,r__I,4e7gUH7IEC,RO12,SEuy,taul,1K8T,lK27,ho_bxgw,NfN9Bz6HFx,J6hvY4Fk1U,smXZ,bDi2,9_Y1,DmlN,VpdQ,sYC_,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,KdSa,ZpHLjE6hJ5,eGzu,beK4AFX,FJ51Im9,cJvF,FzaX,AvcOVQC,oslk,r757j0h,LM8l689qOp,4n2X,xG3x,w_Ub,RAYp,F2FyR07IdsN7I,mj86,-1
34997,0,0,0,0,0,616,0,0,0,8,0,96,0,0,0,0,0,240,300,0,4,88,0,0,200,0,5,0,0,0,0,0,28056,0,0,0,0,2985.64,0,0,...,0,r__I,LDPgUHXZeb,RO12,SEuy,taul,1K8T,TyGl,cSPqFuv,MXlhT5l,pP5FCWS,smXZ,sool,9_Y1,STGZ,sJzTlal,IYzP,me75fM6ugJ,kIsH,uKAI,L84s,NhsEn4L,KdSa,5ZYbB3h,eGzu,mAja5EA,_Xcv,UYBR,FzaX,joS6pd1,oslk,AYu2Oqu,LM8l689qOp,4n2X,kG3k,453m,RAYp,F2FyR07IdsN7I,mj86,-1
34998,0,0,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,166.56,0,5,0,0,0,0,0,0,0,0,0,0,2985.64,0,0,...,0,r__I,xearjIX0Db,RO12,SEuy,taul,1K8T,IvdZ,20dJZJ1,hCdK,yP09M03,smXZ,9XhW,9_Y1,WIr3,VpdQ,IYzP,DHn_WUyBhW_whjA88g9bvA64_,kIsH,uKAI,L84s,CrNX,KdSa,5zARyjR,eGzu

In [83]:
oe = OrdinalEncoder()

In [85]:
df_train = oe.fit_transform(df_train)
df_train = pd.DataFrame(df_train, columns=cols)

# Модель

In [157]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report

In [126]:
X_train = df_train.drop('target', axis=1)
y_train = df_train['target']
cv = StratifiedShuffleSplit(n_splits=7)  # random_state=2179

In [171]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train)

## LogisticRegression

In [131]:
cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

array([0.56417654, 0.56613306, 0.54295884, 0.56910451, 0.56169008,
       0.57750311, 0.56920979])

In [186]:
cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

array([0., 0., 0., 0., 0., 0., 0.])

In [185]:
cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

array([0., 0., 0., 0., 0., 0., 0.])

In [187]:
cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

array([0., 0., 0., 0., 0., 0., 0.])

In [181]:
lr = LogisticRegression(random_state=2179)
lr.fit(X_tr, y_tr)
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      8069
         1.0       0.00      0.00      0.00       681

    accuracy                           0.92      8750
   macro avg       0.46      0.50      0.48      8750
weighted avg       0.85      0.92      0.88      8750



## RidgeClassifier

In [133]:
cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

array([0.69932539, 0.67954255, 0.64283239, 0.65054964, 0.68029251,
       0.68869466, 0.63930734])

In [188]:
cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

array([0., 0., 0., 0., 0., 0., 0.])

In [189]:
cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

array([0., 0., 0., 0., 0., 0., 0.])

In [190]:
cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

array([0., 0., 0., 0., 0., 0., 0.])

In [182]:
rc = RidgeClassifier(random_state=2179)
rc.fit(X_tr, y_tr)
print(classification_report(y_val, rc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      8069
         1.0       0.00      0.00      0.00       681

    accuracy                           0.92      8750
   macro avg       0.46      0.50      0.48      8750
weighted avg       0.85      0.92      0.88      8750



## SGDClassifier

In [135]:
cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

array([0.47591317, 0.49309836, 0.53518244, 0.48979807, 0.48818341,
       0.44814456, 0.56209345])

In [191]:
cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

array([0.0798005 , 0.        , 0.        , 0.09958506, 0.        ,
       0.02649007, 0.        ])

In [184]:
cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

array([0.        , 0.10144928, 0.        , 0.        , 0.06802721,
       0.08219178, 0.        ])

In [192]:
cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

array([0.04214559, 0.03448276, 0.        , 0.02681992, 0.87356322,
       0.        , 0.        ])

In [183]:
sgdc = SGDClassifier(random_state=2179)
sgdc.fit(X_tr, y_tr)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.95      0.13      0.23      8069
         1.0       0.08      0.92      0.15       681

    accuracy                           0.19      8750
   macro avg       0.52      0.53      0.19      8750
weighted avg       0.89      0.19      0.23      8750



### SGDClassifier log loss

In [136]:
cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

array([0.48969161, 0.50299333, 0.45294596, 0.50918937, 0.47953995,
       0.50566905, 0.50738544])

In [193]:
cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

array([0.        , 0.        , 0.03519062, 0.11912226, 0.        ,
       0.13422819, 0.0075188 ])

In [194]:
cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

array([0.05793451, 0.13513514, 0.        , 0.        , 0.15584416,
       0.        , 0.        ])

In [195]:
cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

array([0.        , 0.        , 0.31800766, 0.0651341 , 0.56321839,
       0.        , 0.        ])

In [196]:
sgdc = SGDClassifier(random_state=2179, loss='log')
sgdc.fit(X_tr, y_tr)
print(classification_report(y_val, sgdc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.93      0.85      0.89      8069
         1.0       0.11      0.21      0.14       681

    accuracy                           0.80      8750
   macro avg       0.52      0.53      0.52      8750
weighted avg       0.86      0.80      0.83      8750



## RandomForestClassifier

In [139]:
rfc = RandomForestClassifier(oob_score=True)

In [140]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [142]:
rfc.oob_score_

0.9254

In [143]:
%%time
cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

array([0.62168625, 0.67826442, 0.64490365, 0.67008052, 0.64507576,
       0.66127382, 0.65407527])

In [198]:
cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

array([0., 0., 0., 0., 0., 0., 0.])

In [199]:
cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

array([0., 0., 0., 0., 0., 0., 0.])

In [200]:
cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

array([0., 0., 0., 0., 0., 0., 0.])

In [201]:
rfc = RandomForestClassifier(random_state=2179)
rfc.fit(X_tr, y_tr)
print(classification_report(y_val, rfc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      8069
         1.0       0.00      0.00      0.00       681

    accuracy                           0.92      8750
   macro avg       0.46      0.50      0.48      8750
weighted avg       0.85      0.92      0.88      8750



## GradientBoostingClassifier

In [145]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')

CPU times: user 1.8 s, sys: 213 ms, total: 2.01 s
Wall time: 5min 4s


array([0.72750033, 0.73476867, 0.73284527, 0.70830834, 0.7215527 ,
       0.73489524, 0.72966326])

In [203]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')

CPU times: user 965 ms, sys: 116 ms, total: 1.08 s
Wall time: 5min 6s


array([0.01515152, 0.00763359, 0.        , 0.00754717, 0.01509434,
       0.        , 0.00749064])

In [204]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')

CPU times: user 970 ms, sys: 128 ms, total: 1.1 s
Wall time: 5min 8s


array([0.        , 0.33333333, 0.2       , 0.6       , 0.33333333,
       0.66666667, 0.25      ])

In [205]:
%%time
cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')

CPU times: user 950 ms, sys: 125 ms, total: 1.08 s
Wall time: 5min 6s


array([0.00383142, 0.00383142, 0.00766284, 0.        , 0.01149425,
       0.        , 0.        ])

In [202]:
gbc = GradientBoostingClassifier(random_state=2179)
gbc.fit(X_tr, y_tr)
print(classification_report(y_val, gbc.predict(X_val)))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      8069
         1.0       0.50      0.00      0.01       681

    accuracy                           0.92      8750
   macro avg       0.71      0.50      0.48      8750
weighted avg       0.89      0.92      0.89      8750

